# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will learn how to load and pre-process data from the [COCO dataset](http://cocodataset.org/#home). You will also explore how to design a CNN-RNN model for automatically generating image captions.

This notebook is designed to introduce you to the helper files that you will use to complete your project. 

Note that **any amendments that you make to this notebook will NOT be graded**.

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Instantiate the Data Loader
- [Step 2](#step2): Use the Data Loader to Obtain Batches
- [Step 3](#step3): Tinker with the CNN Encoder
- [Step 4](#step4): Tinker with the RNN Decoder
- [Step 5](#step5): Calculate the Loss
- [Step 6](#step6): Celebrate!

<a id='step1'></a>
## Step 1: Explore the Data Loader

We have already written a [data loader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) that you can use to load the COCO dataset in batches. 

In the code cell below, you will initialize the data loader by using the `get_loader` function in **data_loader.py**.  

> For this project, you are not permitted to change the **data_loader.py** file, which must be used as-is.

The `get_loader` function takes as input a number of arguments that can be explored in **data_loader.py**.  Take the time to explore these arguments now by opening **data_loader.py** in a new window.  For now, we need only focus on four arguments:
1. **`transform`** - an [image transform](http://pytorch.org/docs/master/torchvision/transforms.html) specifying how to pre-process the images and convert them to PyTorch tensors before using them as input to the CNN encoder.  For now, you are encouraged to keep the transform as provided in `transform_train`.  You will have the opportunity later to choose your own image transform to pre-process the COCO images.
2. **`mode`** - one of `'train'` (loads the training data in batches), `'val'` (loads the validation data), or `'test'` (for the test data). We will say that the data loader is in training, validation, or test mode, respectively.
3. **`batch_size`** - determines the batch size.  If training the model, this is total number of image-caption pairs used to amend the model weights in each training step.
4. **`vocab_threshold`** - the total number of times that a word must appear in the in the training captions before it is used as part of the vocabulary.  Images that have fewer than `vocab_threshold` occurrences are considered unknown words. 

We will further detail the `vocab_threshold` and `vocab_from_file` arguments soon.  For now, run the code cell below.  Be patient - it may take a couple of minutes to run!

In [ ]:
from data_loader import get_loader
from torchvision import transforms

# Define a transform to pre-process the training images.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Set the minimum word count threshold.
vocab_threshold = 5

# Specify the batch size.
batch_size = 10

# Obtain the data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold)

When you ran the code cell above, the data loader was stored in the variable `data_loader`.  

You can access the corresponding dataset as `data_loader.dataset`.  This dataset is an instance of the `CoCoDataset` class in **data_loader.py**.  If you are unfamiliar with data loaders and datasets, you are encouraged to review [this PyTorch tutorial](http://pytorch.org/tutorials/beginner/data_loading_tutorial.html).

### Exploring the `__getitem__` Method

The `__getitem__` method in the `CoCoDataset` class determines how a data sample (or *image-caption pair*) is pre-processed before being incorporated into a batch.  When the data loader is in training mode, this method begins by first obtaining the image path (`path`) and caption (`caption`).

#### Image Pre-Processing 

Image pre-processing is relatively straightforward (from the `__getitem__` method in the `CoCoDataset` class):
```python
# Convert image to tensor and pre-process using transform
image = Image.open(os.path.join(self.img_folder, path)).convert('RGB')
image = self.transform(image)
```
After loading the image located at `path`, the image is pre-processed using the same transform that was supplied when instantiating the data loader. 

The pre-processing of the caption is relatively more involved and is covered in the following section.

#### Caption Pre-Processing 

To understand how COCO captions are pre-processed, we'll first need to take a look at the `vocab` instance variable of the `CoCoDataset` class (from the `__init__` method of the `CoCoDataset` class):
```python
def __init__(self, transform, mode, batch_size, vocab_threshold, vocab_file, start_word, 
        end_word, unk_word, annotations_file, vocab_from_file, img_folder):
        ...
        self.vocab = Vocabulary(vocab_threshold, vocab_file, start_word,
            end_word, unk_word, annotations_file, vocab_from_file)
        ...
```
Take the time now to peruse the code to check that `data_loader.dataset.vocab` is an instance of the `Vocabulary` class in **vocabulary.py**.  

We will use an instance of this class to pre-process the COCO captions by mapping tokenized captions to lists of integers (from the `__getitem__` method in the `CoCoDataset` class):

```python
# Convert caption to tensor of word ids.
tokens = nltk.tokenize.word_tokenize(str(caption).lower())   # line 1
caption = []                                                 # line 2
caption.append(self.vocab(self.vocab.start_word))            # line 3
caption.extend([self.vocab(token) for token in tokens])      # line 4
caption.append(self.vocab(self.vocab.end_word))              # line 5
caption = torch.Tensor(caption).long()                       # line 6
```

To see how this code works, we'll apply it to the sample caption in the next code cell.

In [ ]:
sample_caption = 'The golden retriever is running fast.'

In **`line 1`** of the code snippet, every letter in the caption is converted to lowercase, and the [`nltk.tokenize.word_tokenize`](http://www.nltk.org/) function is used to obtain a list of string-valued tokens.  Run the next code cell to visualize the effect on `sample_caption`.

In [ ]:
import nltk

sample_tokens = nltk.tokenize.word_tokenize(str(sample_caption).lower())
print(sample_tokens)

In **`line 2`** and **`line 3`** we initialize an empty list and append an integer that marks the beginning of a caption. 

In [ ]:
sample_caption = []
sample_caption.append(data_loader.dataset.vocab('<start>'))
print(sample_caption)

As you can see above, the integer `0` is always used to mark the beginning of a caption.

Then, in **`line 4`**, we continue the list by adding integers that correspond to each of the tokens in the caption.

In [ ]:
sample_caption.extend([data_loader.dataset.vocab(token) for token in sample_tokens])
print(sample_caption)

In **`line 5`**, we append a final integer to mark the end of the caption.

In [ ]:
sample_caption.append(data_loader.dataset.vocab('<end>'))
print(sample_caption)

As you can see above, the integer `1` is always used to mark the end of a caption. 

Finally, in **`line 6`**, we convert the list of integers to a PyTorch tensor and cast it to [long type](http://pytorch.org/docs/master/tensors.html#torch.Tensor.long).

In [ ]:
import torch

sample_caption = torch.Tensor(sample_caption).long()
print(sample_caption)

add a tl;dr description of the vocabulary.

The total number of tokens can be counted by using the command in the following code cell.  All tokens mapped to a non-negative integer less than the total number of tokens ...

In [ ]:
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

The number of tokens (which we will also refer to as the "size of the vocabulary") is created by looping over the captions in the training dataset.  If a token appears no less than `vocab_threshold` times in the training set, then it is added to the vocabulary.

You will have the option later to amend the `vocab_threshold` argument when instantiating your data loader.  Note that in general, **larger** values for `vocab_threshold` yield a **smaller** number of tokens in the vocabulary.  You are encouraged to check this for yourself now by amending the value of `vocab_threshold` before creating a new data loader.  

In [ ]:
# Modify the minimum word count threshold.
vocab_threshold = 4

# Obtain the data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold)

Use the next code cell to check that the total number of words in the vocabulary has increased.

In [ ]:
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

Note how all tokens that are not in the vocabulary are mapped to the same integer, considered unknown words ...

In [ ]:
print(data_loader.dataset.vocab('jfkafejw'))
print(data_loader.dataset.vocab('ieowoqjf'))

Before training your model, you are encouraged to peruse the `vocabulary.py` file in greater detail.  In the next section, we'll focus on figuring out how to use the data loader to obtain batches of training data.

## Step 2: Use the Data Loader to Obtain Batches

During training, to generate batches of image-caption pairs, we begin by first sampling a caption length.  Then, we retrieve a batch of size `batch_size` with that length.  This approach for assembling batches matches the procedure in [this paper](https://arxiv.org/pdf/1502.03044.pdf), and has been shown to be computationally efficient without degrading performance.

Make clear that they will see this code again when train the model - code is provided for them.

Run the code cell multiple times to see different batches.

In [ ]:
import numpy as np
import torch.utils.data as data

# Randomly sample a caption length, and sample indices with that length.
indices = data_loader.dataset.get_train_indices()
print('sampled indices:', indices)
# Create and assign a batch sampler to retrieve a batch with the sampled indices.
new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
data_loader.batch_sampler.sampler = new_sampler
    
# Obtain the batch.
for batch in data_loader:
    images, captions = batch[0], batch[1]
    break
    
print('images.shape:',images.shape)
print('captions.shape:', captions.shape)

<a id='step3'></a>
## Step 3: Tinker with the CNN Encoder

- make note of module autoreload capability
- need to explain to them what `volatile` is for, etc

In [ ]:
%load_ext autoreload
% autoreload 2

import torch
from torch.autograd import Variable

def to_var(x, volatile=False):
    """ converts a Pytorch Tensor to a variable and moves to GPU if CUDA is available """
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

- provide them a CNN encoder.
- let them decide if they want to swap out the pre-trained architecture for another (make clear this is allowed). show how to do for vgg-16
- consider adding batch normalization if you like ...
- make any modifications to the EncoderCNN class you like. use this as a playground to make sure that your model is working well.
- Your output should be a Torch Variable with shape `[batch_size, embed_size]`.

In [ ]:
from model import EncoderCNN

# TODO: Specify the dimensionality of the image embedding.
embed_size = 256

# Initialize the encoder.
encoder = EncoderCNN(embed_size)
# Move the encoder to GPU if CUDA is available.
if torch.cuda.is_available():
    encoder = encoder.cuda()
    
# Convert last batch of images (from Step 1) to PyTorch Variable.   
images_var = to_var(images, volatile=True)
# Pass the images through the encoder.
features = encoder(images_var)

print('type(features):', type(features))
print('features.shape:', features.shape)

# Check that your encoder satisfies the requirements of the project.
if (type(features)==torch.autograd.variable.Variable) & (features.shape[0]==batch_size) & (features.shape[1]==embed_size):
    print('\nyou may proceed')

<a id='step4'></a>
## Step 4: Tinker with the RNN Decoder

- they have to write the decoder here and I plan to provide a unit test where they can check if implementation is correct
- they have to decide hidden size. note embed_size must be same as with encoder, and output must be same as vocab size.
- if leave notebook and come back, re-run every cell.

In [ ]:
from model import DecoderRNN

# TODO: Specify the number of features in the hidden state of the RNN decoder.
hidden_size = 512

# Store the size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the decoder.
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
# Move the decoder to GPU if CUDA is available.
if torch.cuda.is_available():
    decoder = decoder.cuda()
    
# Convert last batch of captions (from Step 1) to a PyTorch Variable. 
captions_var = to_var(captions)
# Pass the encoder output and captions through the decoder.
outputs = decoder(features, captions_var)

print('type(outputs):', type(outputs))
print('outputs.shape:', outputs.shape)

# Check that your decoder satisfies the requirements of the project.
if (type(outputs)==torch.autograd.variable.Variable) & (outputs.shape[0]==batch_size) & (outputs.shape[1]==captions.shape[1]) & (outputs.shape[2]==vocab_size):
    print('\nyou may proceed')

<a id='step5'></a>
## Step 5: Calculate the Loss

- i give the criterion. students have to calculate the loss
- still trying to figure out how to best assess this. maybe put in py file that's not hidden? tell students not to look unless they really need to? will appear in the next notebook anyway. may as well put it in a file that's not hidden ...

In [ ]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO: Calculate the batch loss.
loss = criterion(outputs.view(-1, vocab_size), captions_var.view(-1))

print('loss:', loss.data[0])